# Preparation

### Connect to Drive and Kaggle

In [ ]:
!pip install -q kaggle
from google.colab import files

files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mackenziecheng","key":"f6a65b977af5878d731672be3d779496"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Install Packages

In [ ]:
# # Core WSI handling
!pip install openslide-python

# # Image processing
!pip install opencv-python Pillow

!pip install tifffile zarr imagecodecs opencv-python pillow pandas



In [48]:
import os
from pathlib import Path
import zipfile
from glob import glob

import tifffile as tiff, zarr
import numpy as np
import cv2, pandas as pd
from PIL import Image

import torch, torchvision as tv
import h5py

import re


## Edge Detection - Contour function

In [ ]:
def _to_uint8(img):
    if img.dtype == np.uint8: return img
    x = img.astype(np.float32); mn, mx = float(x.min()), float(x.max())
    if mx <= mn: return np.zeros_like(x, dtype=np.uint8)
    return ((x - mn) / (mx - mn) * 255.0).astype(np.uint8)




# 通过饱和度分离组织和背景（饱和度高和白色）
# 去掉噪点，让组织区域更连贯
# 找到轮廓线 cv2.findContours
def build_contour_mask_1024(
    lowres_rgb, # 低分辨率彩色图
    mthresh: int = 41,        # 去噪点
    sthresh: int = 12,        # 分离前景背景
    close: int = 2,           # morphology closing kernel
    min_area_fore: int = 12,  # min area (low-res px) 低分辩像素个数
    min_area_hole: int = 8,  # min hole area (low-res px)
    max_n_holes: int = 8,     # cap holes per region 最多保留孔洞数量
):


    img = _to_uint8(lowres_rgb)
    # 组织区域更有颜色，和白色对比强
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV); sat = hsv[..., 1]
    sat = cv2.medianBlur(sat, int(max(1, mthresh) | 1))
    _, bin_s = cv2.threshold(sat, int(sthresh), 255, cv2.THRESH_BINARY)

    # 填补小缝隙
    if close > 0:
        kernel = np.ones((int(close), int(close)), np.uint8)
        bin_s = cv2.morphologyEx(bin_s, cv2.MORPH_CLOSE, kernel)
    contours, hier = cv2.findContours(bin_s, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    if hier is None or len(contours) == 0:
        return (bin_s > 0)
    hier = hier[0]  # [Next, Prev, First_Child, Parent]
    fore_ids = [i for i,h in enumerate(hier) if h[3] == -1]
    kept_fore, holes_per_fore = [], []
    for fid in fore_ids:
        a = cv2.contourArea(contours[fid])
        if a <= 0: continue
        # collect children (holes)
        holes = []
        child = hier[fid][2]
        while child != -1:
            holes.append(child)
            child = hier[child][0]
        hole_areas = [cv2.contourArea(contours[h]) for h in holes]
        real_a = a - (np.sum(hole_areas) if hole_areas else 0.0)
        if real_a >= float(min_area_fore):
            kept_fore.append(fid)
            holes_kept = [h for h in holes if cv2.contourArea(contours[h]) > float(min_area_hole)]
            holes_kept = sorted(holes_kept, key=lambda h: cv2.contourArea(contours[h]), reverse=True)[:max_n_holes]
            holes_per_fore.append(holes_kept)
    H, W = bin_s.shape
    mask = np.zeros((H, W), dtype=np.uint8)
    if kept_fore:
        cv2.drawContours(mask, [contours[i] for i in kept_fore], -1, 255, thickness=cv2.FILLED)
    for holes in holes_per_fore:
        if holes:
            cv2.drawContours(mask, [contours[i] for i in holes], -1, 0, thickness=cv2.FILLED)
    return (mask > 0)


# 是否留下patch
# 组织是否较多？是否有边缘？
def patch_keep(mask_bool, x_m, y_m, w_m, h_m, min_tissue = 0.20, min_edge = 0.05):
    H, W = mask_bool.shape[:2]
    x2, y2 = min(W, x_m + w_m), min(H, y_m + h_m)
    if x_m >= x2 or y_m >= y2: return False, {'cov':0.0,'edge':0.0}
    win = mask_bool[y_m:y2, x_m:x2]
    if win.size == 0: return False, {'cov':0.0,'edge':0.0}
    cov = float(win.mean())
    if cov == 0.0:
        edge_ratio = 0.0
    else:
        eroded = cv2.erode(win.astype(np.uint8), np.ones((3,3), np.uint8), 1).astype(bool)
        border = win ^ eroded
        edge_ratio = float(border.mean())
    keep = (cov >= min_tissue) or (edge_ratio >= min_edge)
    return keep, {'cov': cov, 'edge': edge_ratio}

# 打分数
# score = 0.6 * 组织覆盖率 + 0.4 * edge
def rank_key(stats: dict, alpha: float = 0.6):
    return alpha*float(stats.get('cov',0.0)) + (1.0-alpha)*float(stats.get('edge',0.0))


### Download data

In [38]:
drive_path = '/content/drive/MyDrive/PANDA_OneImage'
os.makedirs(drive_path, exist_ok=True)

image_id = '0005f7aaab2800f6170c399693a96917'
image_filename = f'train_images/{image_id}.tiff'


In [39]:
!kaggle competitions download -c prostate-cancer-grade-assessment -f train.csv -p '{drive_path}'
!kaggle competitions download -c prostate-cancer-grade-assessment -f '{image_filename}' -p '{drive_path}'

train.csv: Skipping, found more recently modified local copy (use --force to force download)
0005f7aaab2800f6170c399693a96917.tiff: Skipping, found more recently modified local copy (use --force to force download)


In [40]:
print(os.listdir(drive_path))

['train.csv', '0005f7aaab2800f6170c399693a96917.tiff', 'patches_20x', 'extracted']


In [51]:
from google.colab import drive
drive.mount('/content/drive')

import os, pathlib
drive_path = '/content/drive/MyDrive/PANDA_OneImage'
image_id = '0005f7aaab2800f6170c399693a96917'
image_filename = f'{image_id}.tiff'
tiff_path = os.path.join(drive_path, image_filename)

print("Exists:", os.path.exists(tiff_path))
print("Size (MB):", round(os.path.getsize(tiff_path)/1024/1024, 2))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Exists: True
Size (MB): 15.38


# Preprocessing

### Import and unzip

*   Many documents implment with OpenSlide, but this Kaggle one does not work as it's a zip --> tifffile and zarr


*   forgetground threshold: reject patches if less than % of pixels are tissues

*   min std: reject patches with low variations (blank)



In [41]:

# with tiff.TiffFile(f'{drive_path}/0005f7aaab2800f6170c399693a96917.tiff') as tf:
#   print(len(tf.series[0].levels))   # number of pyramid levels
#   for lvl in tf.series[0].levels:
#     print(lvl.shape)

In [ ]:
ZIP_OR_TIFF_PATH = f'/content/drive/MyDrive/PANDA_OneImage/{image_id}.tiff'

# output path
EXTRACT_DIR = '/content/drive/MyDrive/PANDA_OneImage/extracted'     # where we unzip and get the real tiff
OUT_DIR     = '/content/drive/MyDrive/PANDA_OneImage/patches_20x'   # where we save patches/CSV, all outputs



# resolution flag
# reso = '20x'
reso = '10x'

Path(EXTRACT_DIR).mkdir(parents=True, exist_ok=True)
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

In [43]:
# unzip the file and return the real tiff
def get_real_tiff(path: str):
    # if file starts with 'PK', it's a ZIP
    with open(path, 'rb') as f:
        sig = f.read(2)
    if sig == b'PK':
        with zipfile.ZipFile(path) as z:
            print('Archive contents:', z.namelist())
            z.extractall(EXTRACT_DIR)
        tiffs = sorted(glob(os.path.join(EXTRACT_DIR, '**', '*.tif*'), recursive=True))
        if not tiffs:
            raise FileNotFoundError('No .tif/.tiff found after extraction.')
        return tiffs[0]
    return path


# zarr: n-dimensional array, like NumPy, but load what you need when you need
# WSIs giant -> zarr
# get the actual array of the pixel data
def _to_zarr_array(znode):
    obj = zarr.open(znode, mode='r')
    # if a single array
    if isinstance(obj, zarr.Array):
        return obj
    # if a group with multi arrays
    if isinstance(obj, zarr.Group):
        keys = list(obj.array_keys())
        if not keys:
            raise ValueError('Zarr Group has no arrays.')
        return obj[keys[0]]
    raise TypeError(f'Unexpected zarr node type: {type(obj)}')


# open the first image in tiff
# collect all levels （pyramid) , 40x --> 20x --> 10x....
def open_pyramid_as_zarr(tf: tiff.TiffFile):

    s0 = tf.series[0]
    arr0 = _to_zarr_array(s0.aszarr())        # level 0 (highest resolution)
    levels = [arr0] + [_to_zarr_array(l.aszarr()) for l in s0.levels]
    # compute downsample factors related to level 0
    # eg: L0 wid = 1000, L1 wid = 500, 1000 / 500 = 2
    downs = [1.0] + [arr0.shape[-2] / lvl.shape[-2] for lvl in levels[1:]]
    return levels, downs

# pick the one closest to my target, 20X here
def pick_level_for_target(downs, target_down=1.0):
    return int(np.argmin([abs(d - target_down) for d in downs]))

# standardize all to RGB unit8 format (H, W, 3)
def ensure_hwc(tile: np.ndarray):
    t = tile
    if t.ndim == 2: # grayscale
        t = np.stack([t]*3, axis=-1)
    elif t.ndim == 3 and t.shape[0] in (3,4) and t.shape[-1] not in (3,4): # (C, H, W)
        t = np.moveaxis(t, 0, -1)  # (C,H,W) -> (H,W,C)
    if t.shape[-1] > 3: # RGBA with alpha
        t = t[..., :3]            # drop alpha
    if t.dtype != np.uint8: # other format
        # best-effort clamp/convert (many WSIs are already uint8)
        t = np.clip(t, 0, 255).astype(np.uint8)
    return t

def save_png(arr: np.ndarray, path: Path):
    Image.fromarray(arr).save(path, format='PNG', compress_level=3)

In [44]:
REAL_TIFF_PATH = get_real_tiff(ZIP_OR_TIFF_PATH)
print('Using TIFF:', REAL_TIFF_PATH)


Archive contents: ['0005f7aaab2800f6170c399693a96917.tiff']
Using TIFF: /content/drive/MyDrive/PANDA_OneImage/extracted/0005f7aaab2800f6170c399693a96917.tiff


In [54]:
with tiff.TiffFile(REAL_TIFF_PATH) as tf:
    series = tf.series[0]
    low = series.levels[-1].asarray()
    img = Image.fromarray(low)

img.save(f"original{image_id}.png")

In [45]:
import tifffile as tiff
with tiff.TiffFile('/content/drive/MyDrive/PANDA_OneImage/extracted/0005f7aaab2800f6170c399693a96917.tiff') as tf:
    print(len(tf.series[0].levels))   # number of pyramid levels
    for lvl in tf.series[0].levels:
        print(lvl.shape)

3
(29440, 27648, 3)
(7360, 6912, 3)
(1840, 1728, 3)


In [ ]:
if reso == '20x':
  # base level: 20X
  hi_level = 0
  PATCH, STRIDE = 1024, 512
  TARGET_DOWN = 1.0

else:
  # 10x level
  hi_level = 1
  PATCH, STRIDE = 512, 256
  TARGET_DOWN = 2.0

In [46]:
with tiff.TiffFile(REAL_TIFF_PATH) as tf:
    levels, downs = open_pyramid_as_zarr(tf)

print('Pyramid downsample factors vs level-0:', [f'{d:.2f}×' for d in downs])

L = pick_level_for_target(downs, TARGET_DOWN)
arrL = levels[L]

# extract H and W
H, W = arrL.shape[-3], arrL.shape[-2]
print(f'Chosen level: {L}  (downsample {downs[L]:.2f}×),  shape≈({H}, {W}, …)')

Pyramid downsample factors vs level-0: ['1.00×', '1.00×', '1.00×', '4.00×']
Chosen level: 0  (downsample 1.00×),  shape≈(7360, 6912, …)


## Run Contour part to get the non-black part

In [ ]:
# choose the lowest reso to build the mask
low_level = len(levels) - 1
arr_low = levels[low_level]
lowres_rgb = np.asarray(arr_low)  # (H_low, W_low, 3) uint8
H_low, W_low = lowres_rgb.shape[:2]



arr0 = levels[hi_level]
H0, W0 = arr0.shape[0], arr0.shape[1]

# Map factor: pixels at hi_level → pixels at low_level
# If your 'downs' is defined as (down from level 0), then:
#   scale_hi_to_low = downs[hi_level] / downs[low_level]
# For hi_level=0 this simplifies to:
scale_hi_to_low = downs[hi_level] / downs[low_level]

# Build the contour mask at the low-res level
contour_mask = build_contour_mask_1024(lowres_rgb)


out_dir = f'patches_out_{reso}'; os.makedirs(out_dir, exist_ok=True)

kept = []
for y in range(0, H0 - PATCH + 1, STRIDE):
    for x in range(0, W0 - PATCH + 1, STRIDE):
        # Map hi-res patch → low-res mask window
        x_m = int(x * scale_hi_to_low)
        y_m = int(y * scale_hi_to_low)
        w_m = max(1, int(PATCH * scale_hi_to_low))
        h_m = max(1, int(PATCH * scale_hi_to_low))

        keep, stats = patch_keep(contour_mask, x_m, y_m, w_m, h_m, min_tissue=0.20, min_edge=0.05)
        if not keep:
            continue

        score = rank_key(stats, alpha=0.6)
        kept.append((score, x, y))

        # Read hi-res tile directly from zarr and save
        tile = np.asarray(arr0[y:y+PATCH, x:x+PATCH, :])
        if tile.shape[:2] != (PATCH, PATCH):
            continue
        Image.fromarray(_to_uint8(tile), 'RGB').save(os.path.join(out_dir, f'p_x{x}_y{y}_s{score:.3f}.png'))

/tmp/ipython-input-2673612727.py:57: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(_to_uint8(tile), 'RGB').save(os.path.join(out_dir, f'p_x{x}_y{y}_s{score:.3f}.png'))


In [37]:

# Folder where your patches are stored
PATCH_DIR = f"patches_out_{reso}"

# Regex to parse filenames like p_x1024_y1024_s0.187.png
pattern = re.compile(r"p_x(\d+)_y(\d+)_s[\d.]+\.png")

# Load all patches and record their coordinates
patches = []
max_x, max_y = 0, 0

for fname in os.listdir(PATCH_DIR):
    match = pattern.match(fname)
    if not match:
        continue
    x, y = int(match.group(1)), int(match.group(2))
    img = Image.open(os.path.join(PATCH_DIR, fname))
    w, h = img.size
    patches.append((x, y, img))

    # Track max extent of the canvas
    max_x = max(max_x, x + w)
    max_y = max(max_y, y + h)

# Create a blank canvas large enough to hold all patches
canvas = Image.new("RGB", (max_x, max_y), (255, 255, 255))

# Paste each patch onto the canvas
for x, y, img in patches:
    canvas.paste(img, (x, y))

# Save the reconstructed image
out_path = f"reconstructed_{reso}.png"
canvas.save(out_path)
print(f"Reconstructed {reso} saved at {out_path}")

Reconstructed 10x saved at reconstructed_10x.png
